In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#MaXViT using stf 
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np
import copy



data_dir = "/kaggle/input/AI-OF-GOD-4/aog_data/train" 
num_classes = 9
batch_size = 16
num_epochs = 10
num_folds = 5
lr = 1e-4
start_fold = 2 #starting at fold 3 as 2 folds were already completed!


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

#Transformations!
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

#Dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)
targets = [label for _, label in full_dataset.imgs]
targets = np.array(targets)

#SKF
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

#Training loop!
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    if fold < start_fold:  #already trained folds chhod do!
        print(f"\n✅ Skipping Fold {fold + 1} (already trained)\n")
        continue

    print(f"\n======================== Fold {fold + 1}/{num_folds} ========================\n")

    # Subsets for this fold
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(copy.deepcopy(full_dataset), val_idx)
    val_subset.dataset.transform = val_transform  #Validation ke transforms!

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2)

    #Model 
    model = models.maxvit_t(weights=models.MaxVit_T_Weights.IMAGENET1K_V1)
    model.classifier[5] = nn.Linear(model.classifier[5].in_features, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_acc = 0.0

    #Epoch loop!
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 40)

        # ---- Training ----
        model.train()
        train_loss, train_preds, train_labels = 0.0, [], []
        for imgs, labels in tqdm(train_loader, desc="Training"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * imgs.size(0)
            train_preds.extend(outputs.argmax(1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_acc = accuracy_score(train_labels, train_preds)
        train_loss /= len(train_subset)

        #Validation!
        model.eval()
        val_loss, val_preds, val_labels = 0.0, [], []
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc="Validation"):
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * imgs.size(0)
                val_preds.extend(outputs.argmax(1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(val_labels, val_preds)
        val_loss /= len(val_subset)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")

        #Best model save krdo!
        if val_acc > best_acc:
            best_acc = val_acc
            save_path = f"best_maxvit_fold_{fold+1}.pth"
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved Best Model for Fold {fold+1} at {save_path}")

    print(f"\n✅ Fold {fold+1} Best Val Accuracy: {best_acc:.4f}\n")

print("🔥 Training completed successfully on single GPU!")

✅ Using device: cuda

✅ Skipping Fold 1 (already trained)


✅ Skipping Fold 2 (already trained)


======================== Fold 3/5 ========================



Downloading: "https://download.pytorch.org/models/maxvit_t-bc5ab103.pth" to /root/.cache/torch/hub/checkpoints/maxvit_t-bc5ab103.pth
100%|██████████| 119M/119M [00:01<00:00, 82.7MB/s] 


Epoch 1/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:19<00:00,  7.73it/s]


Train Loss: 0.6402 | Train Acc: 0.7947
Val Loss:   0.2543 | Val Acc:   0.9196
✅ Saved Best Model for Fold 3 at best_maxvit_fold_3.pth
Epoch 2/10
----------------------------------------


...

Train Loss: 0.0292 | Train Acc: 0.9904
Val Loss:   0.0794 | Val Acc:   0.9767
✅ Saved Best Model for Fold 5 at best_maxvit_fold_5.pth
Epoch 9/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:19<00:00,  7.76it/s]


Train Loss: 0.0183 | Train Acc: 0.9949
Val Loss:   0.0815 | Val Acc:   0.9767
Epoch 10/10
----------------------------------------


Validation: 100%|██████████| 154/154 [00:19<00:00,  7.75it/s]


Train Loss: 0.0156 | Train Acc: 0.9957
Val Loss:   0.0806 | Val Acc:   0.9772
✅ Saved Best Model for Fold 5 at best_maxvit_fold_5.pth

✅ Fold 5 Best Val Accuracy: 0.9772

🔥 Training completed successfully on single GPU!
